In [1]:
# Start writing code here...

# Shopify Data Science Intern Challenge

## Questions from Part 1

On Shopify, we have exactly 100 sneaker shops, and each of these shops sells only one model of shoe. We want to do some analysis of the average order value (AOV). When we look at orders data over a 30 day window, we naively calculate an AOV of $3145.13. Given that we know these shops are selling sneakers, a relatively affordable item, something seems wrong with our analysis. 

Think about what could be going wrong with our calculation. Think about a better way to evaluate this data. What metric would you report for this dataset? What is its value?

In [2]:
#import all of the packages needed for this project

import pandas as pd
##import matplotlib.pyplot as plt
## Found a better graphing package and it worked perfectly for this project
import plotly.express as px

In [3]:
#Let's load the file into the dataframe first

df = pd.read_csv('../Data/datasheet.csv')


Let's take a look at the summary of the dataframe and check on the datatypes of each column.

In [4]:
df.head()

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
0,1,53,746,224,2,cash,2017-03-13 12:36:56
1,2,92,925,90,1,cash,2017-03-03 17:38:52
2,3,44,861,144,1,cash,2017-03-14 4:23:56
3,4,18,935,156,1,credit_card,2017-03-26 12:43:37
4,5,18,883,156,1,credit_card,2017-03-01 4:35:11


In [5]:
df.shape

(5000, 7)

In [6]:
df.dtypes

order_id           int64
shop_id            int64
user_id            int64
order_amount       int64
total_items        int64
payment_method    object
created_at        object
dtype: object

As we can see, there are 5000 rows and 7 columns of data. I do notice that the created_at column type is an object and I would like to convert that to datetime type in case I might need to use it later.

Converting created_at columns to datetime data type in order to query the data....

In [7]:
df['created_at'] = pd.to_datetime(df['created_at'])

Let's check on the type to see if the change is in effect.


In [8]:
df.dtypes

order_id                   int64
shop_id                    int64
user_id                    int64
order_amount               int64
total_items                int64
payment_method            object
created_at        datetime64[ns]
dtype: object

Since the question asked to take the month AOV, I wanted to make sure if the data is indeed from one month and ensure that there are no errors with the data whatsoever. I could just run a query to filter the created_at column


In [9]:
df.query('20170301 < created_at < 20170331')

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
0,1,53,746,224,2,cash,2017-03-13 12:36:56
1,2,92,925,90,1,cash,2017-03-03 17:38:52
2,3,44,861,144,1,cash,2017-03-14 04:23:56
3,4,18,935,156,1,credit_card,2017-03-26 12:43:37
4,5,18,883,156,1,credit_card,2017-03-01 04:35:11
...,...,...,...,...,...,...,...
4995,4996,73,993,330,2,debit,2017-03-30 13:47:17
4996,4997,48,789,234,2,cash,2017-03-16 20:36:16
4997,4998,56,867,351,3,cash,2017-03-19 05:42:42
4998,4999,60,825,354,2,credit_card,2017-03-16 14:51:18


As we can see, after the query there are still 5000 rows of data within the dataframe, which means that the original data contained only data from March of 2017. 

Let's see if there are any duplicated rows within the dataframe.

In [10]:
df.duplicated().sum()

0

There are no duplicated values.

In [11]:
df.nunique()

order_id          5000
shop_id            100
user_id            301
order_amount       258
total_items          8
payment_method       3
created_at        4991
dtype: int64

There are 100 individual shops as shown above, which matches to what the question is given.

We can move on from here to perform our analysis.
Let's take an overview of our data.

In [12]:
df.describe()

,order_id,shop_id,user_id,order_amount,total_items
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000
mean,2500.500000,50.078800,849.092400,3145.128000,8.78720
std,1443.520003,29.006118,87.798982,41282.539349,116.32032
min,1.000000,1.000000,607.000000,90.000000,1.00000
25%,1250.750000,24.000000,775.000000,163.000000,1.00000
50%,2500.500000,50.000000,849.000000,284.000000,2.00000
75%,3750.250000,75.000000,925.000000,390.000000,3.00000
max,5000.000000,100.000000,999.000000,704000.000000,2000.00000


As we can see, the mean for the order_amount is 3145.128, which aligns to the AOV given by the question, however, the value of 3145 seems to high of a value in this case, since each shop only sell one kind of affordable shoes. There could be a possiblity that some of the order amount is very big, indicating that there might exists some wholsale transactions. Let's dive further.

From a logical standpoint, AOV is a measurement to show how an individual shop can improve on increasing its revenues, therefore I think it would be the best if we could calculate AOV for each shop first, We can then calculate an average for all 100 shops if we want.

In [13]:
#calculating AOV for each shop and then take then average
d=[]
group = df.groupby('shop_id')
for key, value in group:
     d.append([key,value['order_amount'].sum()/value['total_items'].sum()])
df1 = pd.DataFrame(d, columns=["shop_id","AOV Per Shop"])
print(df1)

n = len(pd.unique(df['shop_id']))
df1["AOV Per Shop"].sum()/n

##double checking our calculations
##a= df.loc[df['shop_id'] == 1, "order_amount"].sum()
##b = df.loc[df['shop_id'] == 1, "total_items"].sum()
##print(a/b)

    shop_id  AOV Per Shop
0         1         158.0
1         2          94.0
2         3         148.0
3         4         128.0
4         5         142.0
..      ...           ...
95       96         153.0
96       97         162.0
97       98         133.0
98       99         195.0
99      100         111.0

[100 rows x 2 columns]


407.99

AOV came out to be 408, which is still a bit high in my opinion considering the given condition, let's explore further.

In [14]:
#Plotting this result and visualizing
fig = px.box(df1, y="AOV Per Shop")

fig.show()

We see a very obvious outlier in our data, we'll try to show it by writing a function.

In [15]:
def find_outliers_IQR(df1):

   q1=df1.quantile(0.25)

   q3=df1.quantile(0.75)

   IQR=q3-q1

   outliers = df1[((df1<(q1-1.5*IQR)) | (df1>(q3+1.5*IQR)))]

   return outliers


outliers = find_outliers_IQR(df1["AOV Per Shop"])

print("number of outliers: "+ str(len(outliers)))

print("max outlier value: "+ str(outliers.max()))

print("min outlier value: "+ str(outliers.min()))

number of outliers: 2
max outlier value: 25725.0
min outlier value: 352.0


As we can see from both the graph and the returned value from the function above, the most significant outlier in this case is the 25725 AOV value. Such a high value outlier will significantly skew our mean result.

Let's drop the max outlier from our dataset and calculate the AOV again to see what we can get. I'm ignoring the min outlier in this case because although it will have an effect on our AOV, the effect won't be as big as the 25725 value. Let's just keep it in the dataset for now.

In [16]:
df2 = df1[df1['AOV Per Shop'] != 25725]
n = len(pd.unique(df2['shop_id']))
AOV_no = df2["AOV Per Shop"].sum()/n
print(round(AOV_no))

152


Now the AOV for all of the shops came out to be 152, which is more realistic when you put this into the context.

However, during my research for this project, I came across with a blog post from Shopify. The blog included a better option in my opinion than using AOV as the starting point to understand how much a business can grow and maximize the revenue. Here I quote: "Look at your modal, or your most frequent, order values as a starting point for your efforts to increase your overall revenue." This basically aligns with what've discovered so far, a few extreme value will for sure skew our average result and therefore it would be best if we can just look at the most frequent order value and move on from there.



Let's calcualte the mode for AOV Per Shop.

In [17]:
mode_aov = df2["AOV Per Shop"].mode()
print(mode_aov)

mode_aov = df1["AOV Per Shop"].mode()
print(mode_aov)

mean_aov = df1["AOV Per Shop"].mean()
print(mean_aov)

0    153.0
Name: AOV Per Shop, dtype: float64
0    153.0
Name: AOV Per Shop, dtype: float64
407.99


Mode values are the same for both withoutlier dataset and without outlier dataset, this is becasue mode only take account the most frequent order value. We can see how much of a difference this is when comparing mode to mean for the dataset that includes outlier.

# Conclusion for Part 1

I would've use mode AOV as the metric as it reduces the effect of outliers. However, if we choose to perform the above analysis and eliminate the outliers from the original dataset, we could also just use the mean calculated from the without outlier dataset. In this case, I would say the value would be 153 and it would be a good starting point for businesses to consider on how to improve their business to maximize revenue. I hope you enjoyed my analysis.

# Questions from Part 2 SQL

For this question you’ll need to use SQL.

Follow this link to access the data set required for the challenge. Please use queries to answer the following questions. Paste your queries along with your final numerical answers below.

A. How many orders were shipped by Speedy Express in total?

In [18]:
"""
Select Count(OS.OrderID) AS Num_Of_Orders_Shipped_By_SpeedyExpress
From Orders AS OS
Where (Select SH.ShipperID
From Shippers AS SH
Where SH.ShipperName = "Speedy Express") = OS.ShipperID;
"""

'\nSelect Count(OS.OrderID) AS Num_Of_Orders_Shipped_By_SpeedyExpress\nFrom Orders AS OS\nWhere (Select SH.ShipperID\nFrom Shippers AS SH\nWhere SH.ShipperName = "Speedy Express") = OS.ShipperID;\n'

Answer: 54

B. What is the last name of the employee with the most orders? 

In [19]:
"""
SELECT Employees.LastName, COUNT(*) AS NumberOfOrders
FROM Orders 
INNER JOIN Employees ON Orders.EmployeeID = Employees.EmployeeID
GROUP BY Employees.LastName
ORDER BY NumberOfOrders DESC
LIMIT 1;
"""

'\nSELECT Employees.LastName, COUNT(*) AS NumberOfOrders\nFROM Orders \nINNER JOIN Employees ON Orders.EmployeeID = Employees.EmployeeID\nGROUP BY Employees.LastName\nORDER BY NumberOfOrders DESC\nLIMIT 1;\n'

Answer: Last Name : Peacock | Number of Orders: 40

C. What product was ordered the most by customers in Germany?

In [20]:
"""
SELECT Products.ProductName, SUM(OrderDetails.Quantity) AS "TotalOrdered"
FROM Orders JOIN Customers ON Customers.CustomerID = Orders.CustomerID
JOIN OrderDetails ON OrderDetails.OrderID = Orders.OrderID
JOIN Products ON Products.ProductID = OrderDetails.ProductID
WHERE Customers.Country = 'Germany'
GROUP BY OrderDetails.ProductID
ORDER BY TotalOrdered DESC
LIMIT 1;
"""

'\nSELECT Products.ProductName, SUM(OrderDetails.Quantity) AS "TotalOrdered"\nFROM Orders JOIN Customers ON Customers.CustomerID = Orders.CustomerID\nJOIN OrderDetails ON OrderDetails.OrderID = Orders.OrderID\nJOIN Products ON Products.ProductID = OrderDetails.ProductID\nWHERE Customers.Country = \'Germany\'\nGROUP BY OrderDetails.ProductID\nORDER BY TotalOrdered DESC\nLIMIT 1;\n'

Answer: Product Name: Boston Crab Meat | TotalOrdered: 160

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=4b3cda70-c58a-45ea-9815-8e3eb7c2110c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>